In [3]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

model=ChatOpenAI(model="gpt-4o-mini",temperature=0.5)



## Normal chain with stroutparser

In [5]:
parser=StrOutputParser()

prompt=PromptTemplate.from_template("You are helpful assistant! tell me about: {query}")

chain=prompt|model|parser

response=chain.invoke({"query":"GenAI"})
print(response)

Generative AI (GenAI) refers to a class of artificial intelligence models that are designed to generate new content, including text, images, audio, and even video, based on the input they receive. These models learn patterns from existing data and can create new, original outputs that resemble the training data. Here are some key aspects of Generative AI:

### Key Features of Generative AI:

1. **Types of Models**:
   - **Language Models**: These models, like OpenAI's GPT (Generative Pre-trained Transformer), are trained on large text corpora and can generate coherent and contextually relevant text based on prompts.
   - **Image Generators**: Models such as DALL-E and Midjourney can create images from textual descriptions or modify existing images.
   - **Music and Audio**: AI can compose music or generate sound effects based on learned patterns from various music genres or audio samples.

2. **Applications**:
   - **Content Creation**: Generative AI is used for writing articles, creat

## Sequential chain

In [7]:
parser=StrOutputParser()

prompt=PromptTemplate.from_template("You are helpful assistant! tell me about: {query}")
summaryPrompt=PromptTemplate.from_template("You are helpful assistant! summarise this in 5 points and each point should not length more than 15 words: {sumamry}")

chain=prompt|model|parser|summaryPrompt|model|parser

response=chain.invoke({"query":"GenAI"})
print(response)

1. **Content Generation**: GenAI creates text, images, music, and videos indistinguishable from human-made content.

2. **Machine Learning Models**: It uses advanced models like GANs, VAEs, and Transformers for content generation.

3. **Applications**: GenAI is used in entertainment, marketing, healthcare, and art/design industries.

4. **Ethical Considerations**: Issues include copyright, misinformation risks, and potential biases in generated outputs.

5. **Future Trends**: Expect improved content quality, user-friendly tools, and ongoing ethical discussions.


## parallel chains


In [11]:
from langchain_core.runnables import RunnableParallel

parser=StrOutputParser()
summaryPrompt=PromptTemplate.from_template("Give us summary of the moviw in 25 words: {movie}")
reviewPrompt=PromptTemplate.from_template("Give the rating of the movie on the scale of 1 to 10: {movie}")

chain=RunnableParallel({
    "summary":summaryPrompt|model|parser,
    "rating":reviewPrompt|model|parser
})
result = chain.invoke({"movie": "Inception"})

print(result)

{'summary': 'Inception follows a skilled thief who enters dreams to steal secrets. Tasked with planting an idea, he navigates complex dream layers, facing challenges and personal demons.', 'rating': '"Inception," directed by Christopher Nolan, is widely regarded as a masterpiece of science fiction and storytelling. It often receives high ratings from both critics and audiences. On a scale of 1 to 10, it typically scores around 8.5 to 9.5, depending on individual preferences. Many appreciate its complex narrative, visual effects, and thought-provoking themes. Ultimately, the rating can vary based on personal taste!'}


## Conditional Chain

In [20]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.runnables import RunnableBranch

class Schema(BaseModel):
    sentiment: Literal["Positive","Negative","Neutral"]=Field(description="sentiment of the movie review")

structuredModel=model.with_structured_output(Schema)

sentimentPrompt=PromptTemplate.from_template("Give me the review of this movie name: {movie}")

reviewChain=sentimentPrompt|structuredModel

pos_template = PromptTemplate.from_template("""
  The review is postive so write a short appreciation: \n {review}
""")

neg_template = PromptTemplate.from_template("""
  The review is negative so write a short critique: \n {review}
""")

neu_template = PromptTemplate.from_template("""
  The review is neutra so write a short balanced remarks: \n {review}
""")

defult_template = PromptTemplate.from_template("""
  I counln not detect the clear sentimen. Provide a netral response: \n {review}
""")

branch_chain=RunnableBranch(
    (lambda x: x.sentiment=="Positive",pos_template|model|parser),
    (lambda x: x.sentiment=="Negative",neg_template|model|parser),
    (lambda x: x.sentiment=="Neutral",neg_template|model|parser),
    defult_template|model|StrOutputParser
)
final_chain=reviewChain|branch_chain
response=final_chain.invoke({"movie":"Inception"})
print(response)


Thank you for your positive review! We're thrilled to hear that you had a great experience. Your feedback inspires us to keep delivering the best service possible. We appreciate your support!
